In [1]:
##Create the final data set
#Author :Santosh Kumar Udayakumar
#Function : This script loads data from the unzipped sl folder, which is extracted during the data extraction process from the Traffic Lab API. It creates the final dataset by combining Stops and Lines data.

#Load the libraries 

import pandas as pd
import xml.etree.ElementTree as ET



In [2]:
#Inspect the single XML file and identify the meaningful features 
# Parse the XML file
xml_file = 'sl_data/line_1_814_9011001081400000.xml'  # Replace with the path to your XML file
tree = ET.parse(xml_file)
root = tree.getroot()

# Define the namespace
ns = {'netex': 'http://www.netex.org.uk/netex'}

# Extract routes data
routes_data = []
for route in root.findall('.//netex:Route', ns):
    route_id = route.attrib.get('id')
    route_name = route.find('netex:Name', ns).text
    direction = route.find('netex:DirectionType', ns).text
    
    # Extract points in sequence for each route
    points_in_sequence = []
    for point in route.findall('.//netex:PointOnRoute', ns):
        point_id = point.attrib.get('id')
        order = point.attrib.get('order')
        ref = point.find('netex:RoutePointRef', ns).attrib.get('ref')
        points_in_sequence.append({
            'Point_ID': point_id,
            'Order': order,
            'RoutePointRef': ref
        })
    
    routes_data.append({
        'Route_ID': route_id,
        'Route_Name': route_name,
        'Direction': direction,
        'Points_in_Sequence': points_in_sequence
    })

# Create a DataFrame to store routes data
routes_df = pd.DataFrame(routes_data)

# Extract timetable and service journey data along with public and private codes
timetable_data = []
for journey in root.findall('.//netex:ServiceJourney', ns):
    journey_id = journey.attrib.get('id')
    transport_mode = journey.find('netex:TransportMode', ns).text
    
    # Extract operator, day types, and public/private codes
    operator_ref = journey.find('.//netex:OperatorRef', ns).attrib.get('ref')
    day_type_ref = journey.find('.//netex:DayTypeRef', ns).attrib.get('ref')
    
    # Extract LineRef to find public and private codes
    line_ref = journey.find('.//netex:JourneyPatternRef', ns).attrib.get('ref')
    line = root.find(f".//netex:Line[@id='{line_ref}']", ns)
    
    public_code = line.find('netex:PublicCode', ns).text if line is not None else None
    private_code = line.find('netex:PrivateCode', ns).text if line is not None else None
    
    # Extract passing times for each stop in the journey
    for passing_time in journey.findall('.//netex:TimetabledPassingTime', ns):
        stop_ref = passing_time.find('netex:StopPointInJourneyPatternRef', ns).attrib.get('ref')
        arrival_time = passing_time.find('netex:ArrivalTime', ns)
        departure_time = passing_time.find('netex:DepartureTime', ns)
        
        timetable_data.append({
            'Journey_ID': journey_id,
            'TransportMode': transport_mode,
            'OperatorRef': operator_ref,
            'DayTypeRef': day_type_ref,
            'PublicCode': public_code,
            'PrivateCode': private_code,
            'StopPointInJourneyPatternRef': stop_ref,
            'ArrivalTime': arrival_time.text if arrival_time is not None else None,
            'DepartureTime': departure_time.text if departure_time is not None else None
        })

# Create a DataFrame to store service journey timetable data
timetable_df = pd.DataFrame(timetable_data)

# Display the DataFrames
print(routes_df.head())   # Routes DataFrame
print(timetable_df.head()) # Timetable and service journey DataFrame with codes


                         Route_ID         Route_Name Direction  \
0  SE:001:Route:14010000577667337  Södra Hedvigslund  outbound   
1  SE:001:Route:14010000577667012       Gullmarsplan   inbound   

                                  Points_in_Sequence  
0  [{'Point_ID': 'SE:001:PointOnRoute:14010000577...  
1  [{'Point_ID': 'SE:001:PointOnRoute:14010000577...  
                                Journey_ID TransportMode  \
0  SE:001:ServiceJourney:14010000597098249           bus   
1  SE:001:ServiceJourney:14010000597098249           bus   
2  SE:001:ServiceJourney:14010000597098249           bus   
3  SE:001:ServiceJourney:14010000597098249           bus   
4  SE:001:ServiceJourney:14010000597098249           bus   

                        OperatorRef  \
0  SE:001:Operator:9013001003000000   
1  SE:001:Operator:9013001003000000   
2  SE:001:Operator:9013001003000000   
3  SE:001:Operator:9013001003000000   
4  SE:001:Operator:9013001003000000   

                                        

In [34]:
import pandas as pd
import xml.etree.ElementTree as ET
import glob
import os

# Function to extract data and Process all the lines data for time table 
def extract_from_xml(xml_file, private_code):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Define the namespace to simplify searching elements
    ns = {'netex': 'http://www.netex.org.uk/netex'}
    
    # Extract routes data
    routes_data = []
    for route in root.findall('.//netex:Route', ns):
        route_id = route.attrib.get('id')
        route_name = route.find('netex:Name', ns).text
        direction = route.find('netex:DirectionType', ns).text
        
        # Extract points in sequence for each route
        points_in_sequence = []
        for point in route.findall('.//netex:PointOnRoute', ns):
            point_id = point.attrib.get('id')
            order = point.attrib.get('order')
            ref = point.find('netex:RoutePointRef', ns).attrib.get('ref')
            points_in_sequence.append({
                'Point_ID': point_id,
                'Order': order,
                'RoutePointRef': ref
            })
        
        routes_data.append({
            'Route_ID': route_id,
            'Route_Name': route_name,
            'Direction': direction,
            'Points_in_Sequence': points_in_sequence,
            'Private_Code': private_code  # Add private code to routes data
        })

    # Extract timetable and service journey data along with public and private codes
    timetable_data = []
    for journey in root.findall('.//netex:ServiceJourney', ns):
        journey_id = journey.attrib.get('id')
        transport_mode = journey.find('netex:TransportMode', ns).text
        
        # Extract operator, day types, and public/private codes
        operator_ref = journey.find('.//netex:OperatorRef', ns).attrib.get('ref')
        day_type_ref = journey.find('.//netex:DayTypeRef', ns).attrib.get('ref')
        
        # Extract LineRef to find public and private codes
        line_ref = journey.find('.//netex:JourneyPatternRef', ns).attrib.get('ref')
        line = root.find(f".//netex:Line[@id='{line_ref}']", ns)
        
        public_code = line.find('netex:PublicCode', ns).text if line is not None else None
        
        # Extract passing times for each stop in the journey
        for passing_time in journey.findall('.//netex:TimetabledPassingTime', ns):
            stop_ref = passing_time.find('netex:StopPointInJourneyPatternRef', ns).attrib.get('ref')
            arrival_time = passing_time.find('netex:ArrivalTime', ns)
            departure_time = passing_time.find('netex:DepartureTime', ns)
            
            timetable_data.append({
                'Journey_ID': journey_id,
                'TransportMode': transport_mode,
                'OperatorRef': operator_ref,
                'DayTypeRef': day_type_ref,
                'PublicCode': public_code,
                'PrivateCode': private_code,  # Add private code to timetable data
                'StopPointInJourneyPatternRef': stop_ref,
                'ArrivalTime': arrival_time.text if arrival_time is not None else None,
                'DepartureTime': departure_time.text if departure_time is not None else None
            })

    # Create DataFrames
    routes_df = pd.DataFrame(routes_data)
    timetable_df = pd.DataFrame(timetable_data)

    return routes_df, timetable_df

# Main function to read all files starting with 'line'
def process_all_line_files(directory_path):
    all_routes_data = []
    all_timetable_data = []

    # Find all files starting with "line" in the specified directory
    file_pattern = os.path.join(directory_path, 'line*.xml')
    for xml_file in glob.glob(file_pattern):
        print(f"Processing file: {xml_file}")
        
        # Extract private code from the filename (3rd part of the filename)
        private_code = os.path.basename(xml_file).split('_')[2]

        # Extract data from the current XML file, passing the private code
        routes_df, timetable_df = extract_from_xml(xml_file, private_code)
        all_routes_data.append(routes_df)
        all_timetable_data.append(timetable_df)

    # Concatenate all data into single DataFrames
    if all_routes_data:
        final_routes_df = pd.concat(all_routes_data, ignore_index=True)
    else:
        final_routes_df = pd.DataFrame()

    if all_timetable_data:
        final_timetable_df = pd.concat(all_timetable_data, ignore_index=True)
    else:
        final_timetable_df = pd.DataFrame()

    return final_routes_df, final_timetable_df

# Example usage
directory_path = 'sl_data'  # <-- Replace with the actual directory path

# Process all files starting with 'line' in the directory
final_routes_df, final_timetable_df = process_all_line_files(directory_path)

# Display the DataFrames
print(final_routes_df.head())   # Routes DataFrame with private codes
print(final_timetable_df.head()) # Timetable and service journey DataFrame with private codes


Processing file: sl_data/line_1_657_9011001065700000.xml
Processing file: sl_data/line_1_758_9011001075800000.xml
Processing file: sl_data/line_1_752_9011001075200000.xml
Processing file: sl_data/line_1_852_9011001085200000.xml
Processing file: sl_data/line_1_558_9011001055800000.xml
Processing file: sl_data/line_1_552_9011001055200000.xml
Processing file: sl_data/line_1_858_9011001085800000.xml
Processing file: sl_data/line_1_165_9011001016500000.xml
Processing file: sl_data/line_1_76_9011001007600000.xml
Processing file: sl_data/line_1_436_9011001043600000.xml
Processing file: sl_data/line_1_17_9011001001700000.xml
Processing file: sl_data/line_8_3_9011008000300000.xml
Processing file: sl_data/line_8_9_9011008000900000.xml
Processing file: sl_data/line_1_539_9011001053900000.xml
Processing file: sl_data/line_1_833_9011001083300000.xml
Processing file: sl_data/line_1_839_9011001083900000.xml
Processing file: sl_data/line_1_533_9011001053300000.xml
Processing file: sl_data/line_1_480_9

In [35]:
final_routes_df['Route_ID'] = final_routes_df['Route_ID'].str.split(':').str[-1]

In [36]:


# Assuming final_routes_df is your DataFrame

# Expand Points_in_Sequence so that each element of the list becomes a separate row
# Reset the index to avoid reindexing issues
points_expanded = final_routes_df.explode('Points_in_Sequence').reset_index(drop=True)

# Check if Points_in_Sequence has valid data before proceeding
if points_expanded['Points_in_Sequence'].notnull().all():
    # Convert each dictionary in Points_in_Sequence into separate columns
    points_expanded = pd.concat([
        points_expanded.drop(columns=['Points_in_Sequence']),  # Drop original column
        pd.json_normalize(points_expanded['Points_in_Sequence'])  # Normalize nested data
    ], axis=1)

    # Rename the new columns (optional, for clarity)
    points_expanded.rename(columns={
        'Point_ID': 'Point_ID',
        'Order': 'Order',
        'RoutePointRef': 'RoutePointRef'
    }, inplace=True)

    # Display the expanded DataFrame
    print(points_expanded.head())
else:
    print("Some rows in 'Points_in_Sequence' have null values.")


            Route_ID Route_Name Direction Private_Code  \
0  14010000668408490   Vigelsjö  outbound          657   
1  14010000668408490   Vigelsjö  outbound          657   
2  14010000668408490   Vigelsjö  outbound          657   
3  14010000668408490   Vigelsjö  outbound          657   
4  14010000668408490   Vigelsjö  outbound          657   

                                Point_ID Order  \
0  SE:001:PointOnRoute:14010000668408495     1   
1  SE:001:PointOnRoute:14010000668408496     2   
2  SE:001:PointOnRoute:14010000668408497     3   
3  SE:001:PointOnRoute:14010000668408498     4   
4  SE:001:PointOnRoute:14010000668408499     5   

                        RoutePointRef  
0  SE:001:RoutePoint:9022001064146002  
1  SE:001:RoutePoint:9022001062571002  
2  SE:001:RoutePoint:9022001065046001  
3  SE:001:RoutePoint:9022001065042001  
4  SE:001:RoutePoint:9022001064478003  


In [37]:
points_expanded['Point_ID'] = points_expanded['Point_ID'].str.split(':').str[-1]

# Extract numeric part from RoutePointRef
points_expanded['RoutePointRef'] = points_expanded['RoutePointRef'].str.split(':').str[-1]



In [57]:
 points_expanded[points_expanded['RoutePointRef'] == '9021001000814000']

,Route_ID,Route_Name,Direction,Private_Code,Point_ID,Order,RoutePointRef


In [43]:
final_timetable_df['Journey_ID'] = final_timetable_df['Journey_ID'].str.split(':').str[-1]
final_timetable_df['OperatorRef'] = final_timetable_df['OperatorRef'].str.split(':').str[-1]
final_timetable_df['DayTypeRef'] = final_timetable_df['DayTypeRef'].str.split(':').str[-1]
final_timetable_df['StopPointInJourneyPatternRef'] = final_timetable_df['StopPointInJourneyPatternRef'].str.split(':').str[-1]


In [44]:
points_expanded

,Journey_ID,TransportMode,OperatorRef,DayTypeRef,PublicCode,PrivateCode,StopPointInJourneyPatternRef,ArrivalTime,DepartureTime
0,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752564,None,06:09:00
1,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752565,06:09:30,06:09:30
2,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752566,06:10:06,06:10:06
3,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752567,06:11:06,06:11:06
4,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752568,06:11:48,06:11:48
...,...,...,...,...,...,...,...,...,...
1312298,14010000665301478,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14019410000142739_14010000621141281,09:23:00,09:23:00
1312299,14010000665301478,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14019410000142739_14010000621141282,09:23:00,09:23:00
1312300,14010000665301478,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14019410000142739_14010000621141283,09:23:00,09:23:00
1312301,14010000665301478,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14019410000142739_14010000621141284,09:23:00,09:23:00


In [46]:
### Extract stop data 
from lxml import etree

# Load the NeTEx file
with open('sl_data/_stops.xml', 'rb') as file:
    tree = etree.parse(file)

# Define namespaces (if applicable)



namespaces = {
    'netex': 'http://www.netex.org.uk/netex',
    'gml': 'http://www.opengis.net/gml/3.2',
    'siri': 'http://www.siri.org.uk/siri',
    'sam': 'http://www.samtrafiken.se/netex'
}

group_of_stop_places = tree.xpath('//netex:GroupOfStopPlaces', namespaces=namespaces)

stops = []

for group in group_of_stop_places:
    # Use safe retrieval for each element
    name_elem = group.find('netex:Name', namespaces=namespaces)
    description_elem = group.find('netex:Description', namespaces=namespaces)
    private_code_elem = group.find('netex:PrivateCode', namespaces=namespaces)
    latitude_elem = group.find('netex:Centroid/netex:Location/netex:Latitude', namespaces=namespaces)
    longitude_elem = group.find('netex:Centroid/netex:Location/netex:Longitude', namespaces=namespaces)

    group_info = {
        'Group ID': group.get('id'),
        'Name': name_elem.text if name_elem is not None else '',
        'Description': description_elem.text if description_elem is not None else '',
        'Private Code': private_code_elem.text if private_code_elem is not None else '',
        'Latitude': latitude_elem.text if latitude_elem is not None else '',
        'Longitude': longitude_elem.text if longitude_elem is not None else '',
    }
    
    # Append the group info to the data list
    stops.append(group_info)

# Create a DataFrame from the extracted data
stops_df = pd.DataFrame(stops)


In [50]:
stops_df['Group ID'] = stops_df['Group ID'].str.split(':').str[-1]

In [56]:
 stops_df[stops_df['Group ID'] == '9091001000000102']

,Group ID,Name,Description,Private Code,Latitude,Longitude
0,9091001000000102,Styrsvik,Runmarö,102,59.280266,18.731378


In [58]:
from lxml import etree
import pandas as pd

# Load the NeTEx file
with open('sl_data/_stops.xml', 'rb') as file:
    tree = etree.parse(file)

# Define namespaces (if applicable)
namespaces = {
    'netex': 'http://www.netex.org.uk/netex',
    'gml': 'http://www.opengis.net/gml/3.2',
    'siri': 'http://www.siri.org.uk/siri',
    'sam': 'http://www.samtrafiken.se/netex'
}

# Extract all StopPlace elements
stop_places = tree.xpath('//netex:StopPlace', namespaces=namespaces)

# Initialize a list to store stop and quay data
stops = []

for stop_place in stop_places:
    # Extract StopPlace details
    stop_place_id = stop_place.get('id')
    name_elem = stop_place.find('netex:Name', namespaces=namespaces)
    private_code_elem = stop_place.find('netex:PrivateCode', namespaces=namespaces)
    latitude_elem = stop_place.find('netex:Centroid/netex:Location/netex:Latitude', namespaces=namespaces)
    longitude_elem = stop_place.find('netex:Centroid/netex:Location/netex:Longitude', namespaces=namespaces)

    # Create a dictionary for StopPlace info
    stop_info = {
        'StopPlace ID': stop_place_id,
        'StopPlace Name': name_elem.text if name_elem is not None else '',
        'StopPlace Private Code': private_code_elem.text if private_code_elem is not None else '',
        'StopPlace Latitude': latitude_elem.text if latitude_elem is not None else '',
        'StopPlace Longitude': longitude_elem.text if longitude_elem is not None else ''
    }

    # Now check for any Quays within this StopPlace
    quays = stop_place.findall('.//netex:Quay', namespaces=namespaces)
    if quays:
        for quay in quays:
            # Extract Quay details
            quay_id = quay.get('id')
            quay_name_elem = quay.find('netex:Name', namespaces=namespaces)
            quay_private_code_elem = quay.find('netex:PrivateCode', namespaces=namespaces)
            quay_latitude_elem = quay.find('netex:Centroid/netex:Location/netex:Latitude', namespaces=namespaces)
            quay_longitude_elem = quay.find('netex:Centroid/netex:Location/netex:Longitude', namespaces=namespaces)

            # Add Quay details to StopPlace info
            quay_info = {
                'Quay ID': quay_id if quay_id else '',
                'Quay Name': quay_name_elem.text if quay_name_elem is not None else '',
                'Quay Private Code': quay_private_code_elem.text if quay_private_code_elem is not None else '',
                'Quay Latitude': quay_latitude_elem.text if quay_latitude_elem is not None else '',
                'Quay Longitude': quay_longitude_elem.text if quay_longitude_elem is not None else ''
            }

            # Merge StopPlace and Quay info into a single row
            combined_info = {**stop_info, **quay_info}
            stops.append(combined_info)

    else:
        # If no quays exist, just add StopPlace info without quay details
        stops.append(stop_info)

# Create a DataFrame from the extracted data
stops_df = pd.DataFrame(stops)

# Display the DataFrame
print(stops_df.head())


                        StopPlace ID   StopPlace Name StopPlace Private Code  \
0  SE:001:StopPlace:9021001070004000      Alhagsvägen                  70004   
1  SE:001:StopPlace:9021001070004000      Alhagsvägen                  70004   
2  SE:001:StopPlace:9021001070006000          Alby IP                  70006   
3  SE:001:StopPlace:9021001070006000          Alby IP                  70006   
4  SE:001:StopPlace:9021001070012000  Hantverkarvägen                  70012   

  StopPlace Latitude StopPlace Longitude                       Quay ID  \
0          59.234952           17.851566  SE:001:Quay:9022001070004001   
1          59.234952           17.851566  SE:001:Quay:9022001070004002   
2          59.237646           17.851894  SE:001:Quay:9022001070006001   
3          59.237646           17.851894  SE:001:Quay:9022001070006002   
4          59.235563           17.836068                           NaN   

     Quay Name Quay Private Code Quay Latitude Quay Longitude  
0  Alhagsv

In [61]:
stops_df['StopPlace ID'] = stops_df['StopPlace ID'].str.split(':').str[-1]
stops_df['Quay ID'] = stops_df['Quay ID'].str.split(':').str[-1]

In [62]:
stops_df.head()

,StopPlace ID,StopPlace Name,StopPlace Private Code,StopPlace Latitude,StopPlace Longitude,Quay ID,Quay Name,Quay Private Code,Quay Latitude,Quay Longitude
0,9021001070004000,Alhagsvägen,70004,59.234952,17.851566,9022001070004001,Alhagsvägen,1,59.235106,17.851514
1,9021001070004000,Alhagsvägen,70004,59.234952,17.851566,9022001070004002,Alhagsvägen,2,59.234799,17.851618
2,9021001070006000,Alby IP,70006,59.237646,17.851894,9022001070006001,Alby IP,1,59.237838,17.851844
3,9021001070006000,Alby IP,70006,59.237646,17.851894,9022001070006002,Alby IP,2,59.237408,17.851931
4,9021001070012000,Hantverkarvägen,70012,59.235563,17.836068,NaN,NaN,NaN,NaN,NaN


In [64]:
points_expanded.head()

,Route_ID,Route_Name,Direction,Private_Code,Point_ID,Order,RoutePointRef
0,14010000668408490,Vigelsjö,outbound,657,14010000668408495,1,9022001064146002
1,14010000668408490,Vigelsjö,outbound,657,14010000668408496,2,9022001062571002
2,14010000668408490,Vigelsjö,outbound,657,14010000668408497,3,9022001065046001
3,14010000668408490,Vigelsjö,outbound,657,14010000668408498,4,9022001065042001
4,14010000668408490,Vigelsjö,outbound,657,14010000668408499,5,9022001064478003


In [65]:
merged_df = pd.merge(points_expanded, stops_df, how='left', left_on='RoutePointRef', right_on='Quay ID')


In [66]:
merged_df

,Route_ID,Route_Name,Direction,Private_Code,Point_ID,Order,RoutePointRef,StopPlace ID,StopPlace Name,StopPlace Private Code,StopPlace Latitude,StopPlace Longitude,Quay ID,Quay Name,Quay Private Code,Quay Latitude,Quay Longitude
0,14010000668408490,Vigelsjö,outbound,657,14010000668408495,1,9022001064146002,9021001064146000,Campus Roslagen,64146,59.748096,18.685677,9022001064146002,Campus Roslagen,2,59.748264,18.685736
1,14010000668408490,Vigelsjö,outbound,657,14010000668408496,2,9022001062571002,9021001062571000,Stockholmsvägen,62571,59.748736,18.689653,9022001062571002,Stockholmsvägen,2,59.748821,18.689389
2,14010000668408490,Vigelsjö,outbound,657,14010000668408497,3,9022001065046001,9021001065046000,Familjens hus,65046,59.746701,18.691961,9022001065046001,Familjens hus,1,59.746434,18.691735
3,14010000668408490,Vigelsjö,outbound,657,14010000668408498,4,9022001065042001,9021001065042000,Flygfältet,65042,59.748961,18.697201,9022001065042001,Flygfältet,1,59.748912,18.697840
4,14010000668408490,Vigelsjö,outbound,657,14010000668408499,5,9022001064478003,9021001064478000,Ålandsgatan,64478,59.749684,18.703724,9022001064478003,Ålandsgatan,3,59.749747,18.704420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69547,14010000621141260,Vallentuna station,inbound,669,14010000621141281,17,9022001059500001,9021001059500000,Markim,59500,59.590800,18.036978,9022001059500001,Markim,1,59.590748,18.036867
69548,14010000621141260,Vallentuna station,inbound,669,14010000621141282,18,9022001059500001,9021001059500000,Markim,59500,59.590800,18.036978,9022001059500001,Markim,1,59.590748,18.036867
69549,14010000621141260,Vallentuna station,inbound,669,14010000621141283,19,9022001059500001,9021001059500000,Markim,59500,59.590800,18.036978,9022001059500001,Markim,1,59.590748,18.036867
69550,14010000621141260,Vallentuna station,inbound,669,14010000621141284,20,9022001059500001,9021001059500000,Markim,59500,59.590800,18.036978,9022001059500001,Markim,1,59.590748,18.036867


In [67]:
final_timetable_df.head()

,Journey_ID,TransportMode,OperatorRef,DayTypeRef,PublicCode,PrivateCode,StopPointInJourneyPatternRef,ArrivalTime,DepartureTime
0,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752564,None,06:09:00
1,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752565,06:09:30,06:09:30
2,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752566,06:10:06,06:10:06
3,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752567,06:11:06,06:11:06
4,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14019410000161291_14010000664752568,06:11:48,06:11:48


In [68]:
final_timetable_df['StopPointInJourneyPatternRef'] = final_timetable_df['StopPointInJourneyPatternRef'].str.split('_').str[-1]

In [69]:
final_timetable_df.head()

,Journey_ID,TransportMode,OperatorRef,DayTypeRef,PublicCode,PrivateCode,StopPointInJourneyPatternRef,ArrivalTime,DepartureTime
0,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14010000664752564,None,06:09:00
1,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14010000664752565,06:09:30,06:09:30
2,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14010000664752566,06:10:06,06:10:06
3,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14010000664752567,06:11:06,06:11:06
4,14010000666428652,bus,9013001004500000,dfivmm59qm8phnc2c6bahh5tkhnp70hp,None,657,14010000664752568,06:11:48,06:11:48


In [70]:
final_timetable_df.info

<bound method DataFrame.info of                 Journey_ID TransportMode       OperatorRef  \
0        14010000666428652           bus  9013001004500000   
1        14010000666428652           bus  9013001004500000   
2        14010000666428652           bus  9013001004500000   
3        14010000666428652           bus  9013001004500000   
4        14010000666428652           bus  9013001004500000   
...                    ...           ...               ...   
1312298  14010000665301478           bus  9013001004500000   
1312299  14010000665301478           bus  9013001004500000   
1312300  14010000665301478           bus  9013001004500000   
1312301  14010000665301478           bus  9013001004500000   
1312302  14010000665301478           bus  9013001004500000   

                               DayTypeRef PublicCode PrivateCode  \
0        dfivmm59qm8phnc2c6bahh5tkhnp70hp       None         657   
1        dfivmm59qm8phnc2c6bahh5tkhnp70hp       None         657   
2        dfivmm59qm

In [71]:
master_data = pd.merge(merged_df, final_timetable_df, how='left', left_on='Point_ID', right_on='StopPointInJourneyPatternRef')

In [72]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312332 entries, 0 to 1312331
Data columns (total 26 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Route_ID                      1312332 non-null  object
 1   Route_Name                    1312332 non-null  object
 2   Direction                     1312332 non-null  object
 3   Private_Code                  1312332 non-null  object
 4   Point_ID                      1312332 non-null  object
 5   Order                         1312332 non-null  object
 6   RoutePointRef                 1312332 non-null  object
 7   StopPlace ID                  1312332 non-null  object
 8   StopPlace Name                1312332 non-null  object
 9   StopPlace Private Code        1312332 non-null  object
 10  StopPlace Latitude            1312332 non-null  object
 11  StopPlace Longitude           1312332 non-null  object
 12  Quay ID                       1312332 non-

In [73]:
master_data

,Route_ID,Route_Name,Direction,Private_Code,Point_ID,Order,RoutePointRef,StopPlace ID,StopPlace Name,StopPlace Private Code,...,Quay Longitude,Journey_ID,TransportMode,OperatorRef,DayTypeRef,PublicCode,PrivateCode,StopPointInJourneyPatternRef,ArrivalTime,DepartureTime
0,14010000668408490,Vigelsjö,outbound,657,14010000668408495,1,9022001064146002,9021001064146000,Campus Roslagen,64146,...,18.685736,14010000668408800,bus,9013001004500000,62vdge1e16h1c23r2ie5akj4600d2l3r,None,657,14010000668408495,None,06:09:00
1,14010000668408490,Vigelsjö,outbound,657,14010000668408495,1,9022001064146002,9021001064146000,Campus Roslagen,64146,...,18.685736,14010000668409048,bus,9013001004500000,62vdge1e16h1c23r2ie5akj4600d2l3r,None,657,14010000668408495,None,07:08:00
2,14010000668408490,Vigelsjö,outbound,657,14010000668408495,1,9022001064146002,9021001064146000,Campus Roslagen,64146,...,18.685736,14010000668409757,bus,9013001004500000,62vdge1e16h1c23r2ie5akj4600d2l3r,None,657,14010000668408495,None,11:08:00
3,14010000668408490,Vigelsjö,outbound,657,14010000668408495,1,9022001064146002,9021001064146000,Campus Roslagen,64146,...,18.685736,14010000668410654,bus,9013001004500000,62vdge1e16h1c23r2ie5akj4600d2l3r,None,657,14010000668408495,None,16:08:00
4,14010000668408490,Vigelsjö,outbound,657,14010000668408495,1,9022001064146002,9021001064146000,Campus Roslagen,64146,...,18.685736,14010000668411017,bus,9013001004500000,62vdge1e16h1c23r2ie5akj4600d2l3r,None,657,14010000668408495,None,18:08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312327,14010000621141260,Vallentuna station,inbound,669,14010000621141284,20,9022001059500001,9021001059500000,Markim,59500,...,18.036867,14010000665301478,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14010000621141284,09:23:00,09:23:00
1312328,14010000621141260,Vallentuna station,inbound,669,14010000621141285,21,9022001060157006,9021001060157000,Vallentuna station,60157,...,18.078910,14010000630305369,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14010000621141285,19:48:00,None
1312329,14010000621141260,Vallentuna station,inbound,669,14010000621141285,21,9022001060157006,9021001060157000,Vallentuna station,60157,...,18.078910,14010000665310499,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14010000621141285,12:12:00,None
1312330,14010000621141260,Vallentuna station,inbound,669,14010000621141285,21,9022001060157006,9021001060157000,Vallentuna station,60157,...,18.078910,14010000665318371,bus,9013001004500000,bncpagqso7dvr0jmc2o233d3een6g00p,None,669,14010000621141285,14:21:00,None


In [74]:
master_data.to_csv('routes_master_data.csv.gz', index=False, compression='gzip')